In [17]:
#加载必要的库
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms  #torchvison.datasets提供很多公开数据集

In [18]:
#定义超参数  认为定义的参数
BATCH_SIZE = 128 #每批处理的数据
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") #选择训练的设备是GPU还是CPU。
EPOCHS = 10  #训练的轮次

In [19]:
## 构建网络模型
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(1, 10, 5) #1:灰度图片输入通道  10：输出通道(卷积核的个数)  5：kernel卷积核大小 （卷积层）
        self.conv2 = nn.Conv2d(10, 20, 3)#10：输入通道 20：输出通道 3：kernel （卷积层）
        self.fc1 = nn.Linear(20*10*10, 500)#20*10*10：输入通道 500：输出通道 全连接层
        self.fc2 = nn.Linear(500, 10)#500：输入通道 10：输出通道 全连接层 (线性层) 
        self.maxpool1 = nn.MaxPool2d(kernel_size=2,ceil_mode=False)
        self.relu = nn.ReLU() #inplace 是否覆盖input
    
    def forward(self,x):  #前向传播过程
        input_size = x.size(0) #batch_size

        x = self.conv1(x) #输入：batch_size*1*28*28 输出：batch_size*10*24*24 (24 = 28 - 5 + 1) （卷积层）
        x = F.relu(x)  #激活函数，在所有隐藏层之间增加激活函数提高表达能力

        #x = F.max_pool2d(x, 2, 2) #（池化层），对图片进行压缩，输入：bantch*10*24*24 输出：batch*10*12*12  取池化核中最大的值
        x = self.maxpool1(x)
        
        x = self.conv2(x) #输入：batch*10*12*12 输出batch*20*10*10 （10 = 12 - 3 + 1）
        x = F.relu(x)

        x = x.view(input_size, -1) #拉平，-1自动计算维度，20*10*10=2000
        #或者x = torch.flatten(x, 1)

        x = self.fc1(x) #输入：batch*2000 输出：batch*500
        x = F.relu(x)

        x = self.fc2(x) #输入：batch*500 输出：batch*10

        output = F.log_softmax(x,dim=1) #计算分类后，每个数字的概率值

        return output

In [20]:
## 定义训练方法
def train_model(model, device, train_loader, optimizer, epoch):
    #模型训练
    model.train()
    for batch_index, (data, target) in enumerate(train_loader):  #data为一个batch
        #部署到device上
        data, target = data.to(device), target.to(device)
        #初始化梯度
        optimizer.zero_grad()
        #训练后的结果
        output = model(data)
        #计算损失，多分类问题的损失（交叉熵）
        loss = F.cross_entropy(output, target)
        #反向传播（将梯度返回model，优化器利用梯度优化参数）
        loss.backward()
        #参数优化(根据模型中的梯度，进行调优)
        optimizer.step()
        if batch_index % 3000 == 0:
            print(f"Train Epoch:{epoch}\t Loss:{loss.item()}")

In [21]:
## 定义测试方法
def test_model(model, device, test_loader):
    #模型验证
    model.eval()
    #正确率
    correct = 0.0
    #测试损失
    test_loss = 0.0

    with torch.no_grad(): #不计算梯度，不反向传播
        for data, target in test_loader:
            #部署到device
            data, target = data.to(device), target.to(device)
            #测试数据
            output = model(data)
            #计算损失
            test_loss += F.cross_entropy(output, target).item()
            #获取预测值
            pred = output.argmax(dim=1)
            #累计正确的个数
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    print(f"Test Average Loss:{test_loss:.4f}, Accuracy:{correct/len(test_loader.dataset)*100:.3f}")

In [22]:
## 构建pipeline，对图像做处理，transform是pytorch为转换图片提供的工具
pipeline = transforms.Compose([
    transforms.ToTensor(),  #将图片转换为tensor
    transforms.Normalize((0.1307,),(0.3081,)) #正则化降低模型复杂度，解决过拟合
])

In [23]:
## 下载，加载数据
from torch.utils.data import DataLoader

#下载数据集 
#img, target = train_set[0] 获取数据集的内容和标签
#train_set[0] 调用的是__getitem__()默认函数
#test_set.classes[target] 获取标签名称
#train_set和test_set 都是 Dataset类的实例（可以自行重写）  from torch.utils.data import Dataset 
train_set = datasets.MNIST("Data", train=True, download=True, transform=pipeline)

test_set = datasets.MNIST("Data", train=False, download=True, transform=pipeline)

'''img, target = train_set[0]
print(img.shape)
print(train_set.classes[target])'''

#加载数据 
#装载数据时确定了每个batch大小 batch_size变量
#将batch_size个img和target进行打包返回
#train_set和test_set 都是 Dataset类的实例（可以自行重写）
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) 

test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)


In [24]:
##查看 MNIST中文件的图片
'''with open("./Data/MNIST/raw/t10k-images-idx3-ubyte","rb") as f:
    file = f.read()

image1 = [int(str(item).encode("ascii"), 16) for item in file [16 : 16+784]]
import cv2
import numpy as np
image1_np = np.array(image1, dtype=np.uint8).reshape(28,28,1)

print(image1_np.shape)
cv2.imwrite("digit.jpg", image1_np)'''

'with open("./Data/MNIST/raw/t10k-images-idx3-ubyte","rb") as f:\n    file = f.read()\n\nimage1 = [int(str(item).encode("ascii"), 16) for item in file [16 : 16+784]]\nimport cv2\nimport numpy as np\nimage1_np = np.array(image1, dtype=np.uint8).reshape(28,28,1)\n\nprint(image1_np.shape)\ncv2.imwrite("digit.jpg", image1_np)'

In [25]:
 ## 定义优化器
model = Net().to(DEVICE) ##模型实例化

optimizer = optim.Adam(model.parameters()) ##调节模型参数的优化器

In [26]:
for epoch in range(1, EPOCHS+1):
    train_model(model, DEVICE, train_loader, optimizer, epoch)
    test_model(model,DEVICE,test_loader)

Train Epoch:1	 Loss:2.308753490447998
Test Average Loss:0.0005, Accuracy:97.860
Train Epoch:2	 Loss:0.04120075702667236
Test Average Loss:0.0004, Accuracy:98.460
Train Epoch:3	 Loss:0.02646901085972786
Test Average Loss:0.0003, Accuracy:98.730
Train Epoch:4	 Loss:0.023419199511408806
Test Average Loss:0.0003, Accuracy:98.830
Train Epoch:5	 Loss:0.0029025208204984665
Test Average Loss:0.0003, Accuracy:98.870
Train Epoch:6	 Loss:0.008113086223602295
Test Average Loss:0.0002, Accuracy:99.110
Train Epoch:7	 Loss:0.00480921333655715
Test Average Loss:0.0002, Accuracy:99.000
Train Epoch:8	 Loss:0.007507923990488052
Test Average Loss:0.0003, Accuracy:98.800
Train Epoch:9	 Loss:0.0014581014402210712
Test Average Loss:0.0003, Accuracy:99.020
Train Epoch:10	 Loss:0.01858961209654808
Test Average Loss:0.0003, Accuracy:98.900
